<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-러쉬" data-toc-modified-id="1.-러쉬-1">1. 러쉬</a></span></li><li><span><a href="#2.-아로마티카-(X)" data-toc-modified-id="2.-아로마티카-(X)-2">2. 아로마티카 (X)</a></span></li><li><span><a href="#3.-클레어스" data-toc-modified-id="3.-클레어스-3">3. 클레어스</a></span></li></ul></div>

러쉬, 아로마티카, 클레어스, 이든 미네랄, 오엠, adorable, sephora

(+ 마녀공장, 스킨푸드, 아떼, 마켓컬리, so natural, is green, 오드리앤영)

In [1]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings('ignore')

# 1. 러쉬

In [41]:
url = 'https://lush.co.kr/goods/goods_list.php?cateCd=001019001'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [11]:
# 소분류명 수집
so_cat = html.find_all('span', class_='on')
print(len(so_cat))
so_cat

1


[<span class="on"><a href="?cateCd=001019001">배쓰</a> <em>(72)</em> </span>]

In [17]:
import re

re.compile('[가-힣]+').findall(so_cat[0].text)[0]

'배쓰'

In [4]:
# 상품명 수집
item = html.find_all('span', class_='prdName')
print(len(item))
item[:5]

72


[<span class="prdName">킵 잇 플러피 퍼퓸 버블 보틀</span>,
 <span class="prdName">카마 퍼퓸 버블 보틀</span>,
 <span class="prdName">디어 존 퍼퓸 버블 보틀</span>,
 <span class="prdName">더티 퍼퓸 버블 보틀</span>,
 <span class="prdName">러브 퍼퓸 버블 보틀</span>]

In [6]:
# 가격 수집 (정가)
price = html.find_all('span', class_='cost')
print(len(price))
price[:5]

72


[<span class="cost">
 <strong>￦ 23,000</strong> </span>,
 <span class="cost">
 <strong>￦ 35,000</strong> </span>,
 <span class="cost">
 <strong>￦ 35,000</strong> </span>,
 <span class="cost">
 <strong>￦ 23,000</strong> </span>,
 <span class="cost">
 <strong>￦ 35,000</strong> </span>]

In [9]:
price[0].text.replace('\n','').strip()

'￦ 23,000'

In [10]:
# 러쉬는 할인을 안 해..

In [18]:
# 최종 스크래핑 코드
def cosmetic_scraping(category):
    
    total_data = []
    for cat in category:
        url = f'https://lush.co.kr/goods/goods_list.php?cateCd={cat}'
        html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
        
        # 소분류명, 상품명, 가격 (정가, 할인가)
        so_cat = html.find_all('span', class_='on')
        so_cat2 = re.compile('[가-힣]+').findall(so_cat[0].text)[0]
        item = html.find_all('span', class_='prdName')
        price = html.find_all('span', class_='cost')

        data = []
        for i in range(len(item)):
            item2 = item[i].text
            price2 = price[i].text.replace('\n','').strip()
            data.append([so_cat2, item2, price2])
        total_data.append(data)
    
    df = pd.DataFrame(sum(total_data, []))
    df.columns = ['소분류명', '상품명', '정가']
    return df

In [34]:
# 카테고리 선정 후 수집
df = cosmetic_scraping(['001019001','001019002','001019003','001019004','001019005','001019006','001019007'])
print(df.shape)
df.head()

(352, 3)


,소분류명,상품명,정가
0,배쓰,킵 잇 플러피 퍼퓸 버블 보틀,"￦ 23,000"
1,배쓰,카마 퍼퓸 버블 보틀,"￦ 35,000"
2,배쓰,디어 존 퍼퓸 버블 보틀,"￦ 35,000"
3,배쓰,더티 퍼퓸 버블 보틀,"￦ 23,000"
4,배쓰,러브 퍼퓸 버블 보틀,"￦ 35,000"


In [36]:
df['브랜드명'], df['할인가'] = '러쉬', ' '
df = df[['브랜드명','소분류명','상품명','정가','할인가']]
df['정가'] = df['정가'].apply(lambda x: x.replace('￦ ','') + '원')
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(352, 5)


,브랜드명,소분류명,상품명,정가,할인가
0,러쉬,배쓰,킵 잇 플러피 퍼퓸 버블 보틀,"23,000원",
1,러쉬,배쓰,카마 퍼퓸 버블 보틀,"35,000원",
2,러쉬,배쓰,디어 존 퍼퓸 버블 보틀,"35,000원",
3,러쉬,배쓰,더티 퍼퓸 버블 보틀,"23,000원",
4,러쉬,배쓰,러브 퍼퓸 버블 보틀,"35,000원",


In [37]:
df.to_csv('./cosmetic/러쉬.csv', index=False)

In [39]:
df[df.상품명.duplicated()]

,브랜드명,소분류명,상품명,정가,할인가
54,러쉬,배쓰,레인보우,"20,000원",
71,러쉬,배쓰,더 컴포터,"18,000원",
80,러쉬,샤워,슬리피,"8,000원",
92,러쉬,샤워,레인보우,"20,000원",
93,러쉬,샤워,몬스터스 & 에일리언스,"19,000원",
110,러쉬,보디,팅글,"34,000원",
111,러쉬,보디,팅글 225g,"45,000원",
116,러쉬,보디,오션 솔트 셀프-프리저빙 120g/250g,"27,000원",
131,러쉬,보디,버피 90g,"17,000원",
149,러쉬,페이스,오션 솔트 셀프-프리저빙 120g/250g,"27,000원",


# 2. 아로마티카 (X)

In [44]:
url = 'https://aromatica.co.kr/product/search.html?keyword=%EB%B9%84%EA%B1%B4&x=26&y=16&keyword_layer=&except_keyword=#none'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [46]:
# 상품명 수집
item = html.find_all('span', style='font-size:15px;color:#000000;')
print(len(item))
item[:5]

62


[<span style="font-size:15px;color:#000000;">컨셔스 비건 립 컬러 엘더베리</span>,
 <span style="font-size:15px;color:#000000;">컨셔스 비건 립 컬러 비터체리</span>,
 <span style="font-size:15px;color:#000000;">컨셔스 비건 립 컬러 애프리콧</span>,
 <span style="font-size:15px;color:#000000;">"과즙미 듬뿍" 컨셔스 비건 립 컬러 세트</span>,
 <span style="font-size:15px;color:#000000;">"광채촉촉" 로즈 스킨케어 패키지 (듀얼 화장솜(80매입) + 비건 립 컬러 1종 증정)</span>]

In [53]:
# 가격 수집 (정가)
price = html.find_all('span', style='font-size:13px;color:#000000;')
print(len(price))
price[:5]

97


[<span style="font-size:13px;color:#000000;">판매가</span>,
 <span style="font-size:13px;color:#000000;">20,000 won</span>,
 <span style="font-size:13px;color:#000000;">판매가</span>,
 <span style="font-size:13px;color:#000000;">20,000 won</span>,
 <span style="font-size:13px;color:#000000;">판매가</span>]

# 3. 클레어스